# ETL:  Parámetros

## Librerias

In [1]:
# Generales
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings('ignore')

# Funciones Snowflake
import funciones as snow_func

In [2]:
# Aumentar número de columnas que se pueden ver
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.10f}'.format
# Cada columna será tan grande como sea necesario para mostrar todo su contenido
pd.set_option('display.max_colwidth', 0)

### Snowflake

In [3]:
# Librerias necesarias para subir a Snowflake
import os
import json
import snowflake.connector # [pip install snowflake-connector-python]
from snowflake.connector.pandas_tools import write_pandas # [pip install "snowflake-connector-python[pandas]"]
from snowflake.snowpark import Session

In [4]:
# Paso 1: Definir la ruta al archivo JSON en el escritorio
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop\Conn")
json_file_path = os.path.join(desktop_path, "snowflake_credentials.json")

# Paso 2: Leer las credenciales desde el archivo JSON
with open(json_file_path, 'r') as file:
    credentials = json.load(file)

# Paso 3: Definir los parámetros de conexión usando las credenciales
connection_parameters = {
        "account": credentials["ACCOUNT_SNOWFLAKE"],
        "user": credentials["USER_SNOWFLAKE"],
        "password": credentials["PASSWORD_SNOWFLAKE"],
        "role": credentials["ROLE_SNOWFLAKE"],
        "warehouse": credentials["WAREHOUSE"]
    }

# Paso 5: Crear un objeto de conexión utilizando snowflake.connector
session = Session.builder.configs(connection_parameters).create()
print("Sesión actual:", {session})

Sesión actual: {<snowflake.snowpark.session.Session object at 0x00000277E46A6AD0>}


In [5]:
# Crear objeto de conexión
conn = session.connection

## 1. Parámetros

In [6]:
# Solo se debe cambiar la ubicación del archivo
path_parametros = "C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/PARAMETROS/"
parametros_file = 'Param.xlsx'

In [7]:
# Leer parámetros
df_param = pd.read_excel(path_parametros + parametros_file, sheet_name='Hoja1', dtype=str)
df_param

,Parametro,Valor,Eje
0,Fecha de actualización,12/12/2024,Transversal
1,Año cerrado (T-1),2022,Exportaciones
2,Año cerrado (T),2023,Exportaciones
3,Año corrido (T-1),2023(Ene-Oct),Exportaciones
4,Año corrido (T),2024(Ene-Oct),Exportaciones
5,Año corrido texto (T-1),Ene-Oct 2023,Exportaciones
6,Año corrido texto (T),Ene-Oct 2024,Exportaciones
7,Mes corrido texto (T),Octubre,Exportaciones
8,Año cerrado (T-1),2022,Inversión
9,Año cerrado (T),2023,Inversión


## 2. Subir a Snowflake

##### Usar base de datos y esquema para análisis

In [8]:
# Usar base de datos:
sql_database = """
USE DATABASE DOCUMENTOS_COLOMBIA;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_database)

,status
0,Statement executed successfully.


In [9]:
# Usar esquema PARAMETROS:
sql_schema_param = """
USE SCHEMA PARAMETROS;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_param)

,status
0,Statement executed successfully.


In [10]:
# Asegurar que estamos en la ubicación que se desea para subir las bases de datos
ubicacion = "SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;"
# Ejecutar
snow_func.snowflake_sql(conn, ubicacion)

,WAREHOUSE,DATABASE,SCHEMA
0,WH_PROCOLOMBIA_ANALITICA,DOCUMENTOS_COLOMBIA,PARAMETROS


##### SUBIR PARÁMETROS

In [11]:
# Subir df_param
# Ejecutar
snow_func.snowflake_cargar_df(conn, df_param, 'PARAMETROS')

'DataFrame cargado exitosamente en la tabla: 21 filas en 1 chunks.\nTiempo de carga: 4.13 segundos.\nProceso terminado'

In [12]:
# Verificar que se cargó correctamente:
snow_func.snowflake_sql(conn, "SELECT COUNT (*) FROM PARAMETROS;")

,COUNT (*)
0,21


In [13]:
# Mismas filas
df_param.shape

(21, 3)

## 3. Cerrar sesión, conexión y cursor

In [14]:
conn.close()
session.close()